<a href="https://colab.research.google.com/github/jefsbruno/-JSON-Server-Fake-API/blob/master/pipeline_etl_santander.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

# ***E***xtract

Extrai a lista de IDS de usuário a partir do arquivo CSV. Para cada Id, faça uma requisição GET para obter os dados do usuario correspondente.

In [ ]:
import pandas as pd

df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

In [ ]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

# **T**ransform

Utilize a API do OpenAi GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.

In [ ]:
!pip install openai

In [ ]:
openai_api_key = 'TODO'

import openai

openai.api_key = openai_api_key

def generate_ai_news(user):
  completion = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
      {
          "role": "system",
          "content": "Você é um especialista em markting bancário."
      },
      {
          "role": "user",
          "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)"
      }
    ]
  )
  return completion.choices[0].message.content.strip('\"')

for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })

# ***L***oad

Atualize a lista de "news" de cada usuario na APi com a nova mensagem gerada

In [ ]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")